## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-24-03-39-58 +0000,nyt,Where U.S. Medicines Are Made and How Trump’s ...,https://www.nytimes.com/2025/08/23/health/pres...
1,2025-08-24-03-33-37 +0000,nyt,Fed Officials Try to Keep Focus on Economy as ...,https://www.nytimes.com/2025/08/23/business/po...
2,2025-08-24-03-33-24 +0000,nyt,5 Takeaways From Ghislaine Maxwell’s Interview...,https://www.nytimes.com/2025/08/22/us/politics...
3,2025-08-24-03-13-01 +0000,latimes,Photos: An epic day in the ocean,https://www.latimes.com/california/story/2025-...
4,2025-08-24-03-00-00 +0000,wsj,Russia Covets This Ukraine Province Above All....,https://www.wsj.com/world/russia/russia-covets...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,24
258,menendez,11
65,new,8
452,california,7
261,parole,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
88,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...,70
94,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...,60
76,2025-08-23-15-46-51 +0000,nypost,Macron says Trump’s ‘unpredictability’ key ass...,https://nypost.com/2025/08/23/world-news/macro...,39
112,2025-08-23-09-02-10 +0000,nyt,"Lisa Cook, Who Broke Ground at the Fed, Faces ...",https://www.nytimes.com/2025/08/23/business/li...,37
116,2025-08-23-09-01-36 +0000,nyt,Harvard Is Making Changes Trump Officials Want...,https://www.nytimes.com/2025/08/23/us/harvard-...,36


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
88,70,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...
128,34,2025-08-23-07-07-23 +0000,nypost,Family of missing Texas woman Kaura Taylor who...,https://nypost.com/2025/08/23/world-news/texas...
130,33,2025-08-23-06-09-10 +0000,wapo,"Lyle Menendez, who killed his parents, denied ...",https://www.washingtonpost.com/nation/2025/08/...
8,29,2025-08-24-02-00-00 +0000,wsj,Bashing the president’s tariff policies could ...,https://www.wsj.com/politics/policy/gop-tariff...
92,26,2025-08-23-12-42-42 +0000,cbc,U.S. seeks to deport Kilmar Abrego Garcia to U...,https://www.cbc.ca/news/world/kilmar-abrego-ga...
37,24,2025-08-23-21-31-00 +0000,wsj,The Pentagon has for months been blocking Ukra...,https://www.wsj.com/politics/national-security...
54,22,2025-08-23-19-36-55 +0000,nypost,FBI believes Bolton should face charges over a...,https://nypost.com/2025/08/23/us-news/fbi-beli...
46,22,2025-08-23-20-43-29 +0000,nypost,Zohran Mamdani posts cheeky nod to ex-Adams ad...,https://nypost.com/2025/08/23/us-news/zohran-m...
64,22,2025-08-23-18-30-32 +0000,nypost,"North Korea builds secret military base, incre...",https://nypost.com/2025/08/23/world-news/north...
110,21,2025-08-23-09-30-00 +0000,wsj,"At Jackson Hole, Fed Chair Jerome Powell deliv...",https://www.wsj.com/economy/central-banking/po...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
